In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer, DataProcess, CovarianceStructure, ObjectiveLib
#import matplotlib.pyplot as plt
import pickle 

In [2]:
Nt = 1

# they are set up but not work
max_manual_num = 100
min_interval_num = 100

In [3]:
#measure_names = ["y_CO2", "C_CO2", "C_N2", "Tg", "Pg", "vg", "q_CO2","Ts", "y_theta"]
measure_names = ["Tg", "Pg", "vg", "Ts", "y_CO2",  "C_CO2", "C_N2",  "q_CO2","y_theta"]
num_measure = len(measure_names)

#z_pos = 8
#z_names = "01234567"
z_pos = 3
z_names = "012"

o_pos = 3 
o_names = "012"

num_mod = 2 
mod_names = "ads", "des"

num_total = num_measure*z_pos*o_pos*num_mod

all_ind = list(range(num_total))

print(num_total)

162


In [4]:
# name generator
# CCO2, ADS, Z=0, O=0
# CCO2, ADS, Z=1, O=0
# ...
# CCO2, ADS, Z=0, O=1
#...
# CCO2, DES, Z=0, O=0
# CCO2, DES, Z=1, O=0
# ...
# CCO2, DES, Z=0, O=1
# adsorption + desorption, ...

def name_generator():
    
    name_list = []
    static_costs = []
    dynamic_costs = []
    
    
    for m_n in measure_names:
        for mod_n in mod_names:
            if m_n in ["Tg", "Pg", "vg", "Ts"]:
                static = 1000
                dynamic = 0 
            else:
                static = 0 
                dynamic = 100 
            
            for o in list(o_names):
                for z in list(z_names):
                    name = "_".join([m_n, z, o, mod_n])
                    
                    name_list.append(name)
                    static_costs.append(static)
                    dynamic_costs.append(dynamic)
    
    return name_list, static_costs, dynamic_costs

all_names, static_cost, dynamic_cost = name_generator() 
#print(all_names)

In [5]:
num_z_o = z_pos*o_pos     

max_manual = [max_manual_num]*num_total
min_time_interval = [min_interval_num]*num_total

In [6]:
measure_info = pd.DataFrame({
    "name": all_names,
    "Q_index": all_ind,
        "static_cost": static_cost,
    "dynamic_cost": dynamic_cost,
    "min_time_interval": min_time_interval, 
    "max_manual_number": max_manual
})

print(measure_info)

                name  Q_index  static_cost  dynamic_cost  min_time_interval  \
0         Tg_0_0_ads        0         1000             0                100   
1         Tg_1_0_ads        1         1000             0                100   
2         Tg_2_0_ads        2         1000             0                100   
3         Tg_0_1_ads        3         1000             0                100   
4         Tg_1_1_ads        4         1000             0                100   
..               ...      ...          ...           ...                ...   
157  y_theta_1_1_des      157            0           100                100   
158  y_theta_2_1_des      158            0           100                100   
159  y_theta_0_2_des      159            0           100                100   
160  y_theta_1_2_des      160            0           100                100   
161  y_theta_2_2_des      161            0           100                100   

     max_manual_number  
0                  100  
1

In [7]:
num_static = 4*num_mod*z_pos*o_pos
num_dynamic  = 5*num_mod*z_pos*o_pos

print(num_static, num_dynamic, num_total)
num_total = num_static+num_dynamic

72 90 162


In [8]:
dataObject = DataProcess()
dataObject.read_jacobian('./MO_QVs/Q_z3')
Q = dataObject.get_Q_list(list(range(num_static)), list(range(num_static, num_total)), Nt)

print(np.shape(Q))

print(Q[1])

(162, 1, 5)
[[0.01424043941256059, -0.09380282162396725, -0.06041254277988628, 0.04913281242124867, -0.0004855924942148704]]


In [ ]:
print(Q[72])

print(data[72])

q1 = np.asarray(Q[72]).reshape(1,5)
q2 = np.asarray(Q[72]).reshape(5,1)

#print(q1)
#print(q2)
v = data[72]

test = 1/v*(q2@q1)

print(test)

In [ ]:
file1 = open("./MO_QVs/y_CO2_0", "rb")
realq = pickle.load(file1)
file1.close()

file2 = open("./MO_QVs/Var_y_CO2_0", "rb")
realvar = pickle.load(file2)
file2.close()

print(realq["ads_y_CO2_0"][5])
print(realvar["ads_y_CO2_0"][5])

In [9]:
## error covariance 
file = open("./MO_QVs/Var_z3", "rb")
data = pickle.load(file)
file.close()

error_cov = np.zeros((num_total, num_total))

for i in range(num_total):
    error_cov[i,i] = data[i]

In [10]:
calculator = MeasurementOptimizer(Q, measure_info, error_cov=error_cov, error_opt=CovarianceStructure.measure_correlation, verbose=True)


fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

Covariances are between measurements at the same time.
(26244, 5, 5)


In [11]:
#init_cov_y = np.zeros((num_total, num_total))

#for i in range(num_static, num_total):
#    for j in range(num_static, num_total):
#        init_cov_y[i,j]=1



#fim_prior = [[0]*5 for _ in range(5)]

#for _ in range(5):
#    fim_prior[_][_] = 1 
    
#fim_prior = np.asarray(fim_prior)

In [45]:
with open('Sep17_7000_a', 'rb') as f:
    init_cov_y = pickle.load(f)
print(init_cov_y)

with open('Sep17_FIM_7000_a', 'rb') as f:
    fim_prior = pickle.load(f)
print(fim_prior)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[ 4.28232381e+05  3.98880392e+04 -5.32513262e+05  3.30957645e+05
  -9.14181477e+03]
 [ 3.98880392e+04  3.01738037e+04 -9.16959803e+04  6.70499642e+04
  -9.29915101e+02]
 [-5.32513262e+05 -9.16959803e+04  3.36200285e+08 -2.77508537e+08
   1.26432160e+06]
 [ 3.30957645e+05  6.70499642e+04 -2.77508537e+08  2.29943803e+08
  -1.02991666e+06]
 [-9.14181477e+03 -9.29915101e+02  1.26432160e+06 -1.02991666e+06
   6.33701593e+03]]


In [46]:
mip_option = False
objective = ObjectiveLib.A
sparse_opt = True
fix_opt = False

manual_num = 100
budget_opt = 8000

total_manual_init = int(budget_opt//100)
dynamic_install_init = [1]*num_total 

num_dynamic_time = [0]

#static_dynamic = [[0,3],[1,4],[2,5]]
#time_interval_for_all = True

#dynamic_time_dict = {}
#for i, tim in enumerate(num_dynamic_time[1:]):
#    dynamic_time_dict[i] = np.round(tim, decimals=2)
    
#print(dynamic_time_dict)

In [47]:
mod = calculator.continuous_optimization(mixed_integer=mip_option, 
                      obj=objective, 
                    fix=fix_opt, 
                    upper_diagonal_only=sparse_opt, 
                    num_dynamic_t_name = num_dynamic_time, 
                    manual_number = manual_num, 
                    budget=budget_opt,
                    init_cov_y= init_cov_y,
                    initial_fim = fim_prior,
                    dynamic_install_initial = dynamic_install_init, 
                    total_manual_num_init=total_manual_init)

mod = calculator.solve(mod,  mip_option=mip_option, objective = objective, tol=1e-6)

72
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
72
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
73
[1, 1, 1, 1, 1, 1

Ipopt 3.13.2: linear_solver=ma57
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
   

In [48]:
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(i,5):
        fim_result[i,j] = fim_result[j,i] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))
print(np.linalg.eigvals(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

ans_y, sol_y = calculator.extract_solutions(mod)
print('pyomo calculated cost:', pyo.value(mod.cost))

[[ 4.28240987e+05  3.98923760e+04 -5.32508854e+05  3.30955595e+05
  -9.14186822e+03]
 [ 3.98923760e+04  3.01769946e+04 -9.17057602e+04  6.70590081e+04
  -9.29983554e+02]
 [-5.32508854e+05 -9.17057602e+04  3.36200461e+08 -2.77508684e+08
   1.26432220e+06]
 [ 3.30955595e+05  6.70590081e+04 -2.77508684e+08  2.29943926e+08
  -1.02991717e+06]
 [-9.14186822e+03 -9.29983554e+02  1.26432220e+06 -1.02991717e+06
   6.33701851e+03]]
trace: 566609142.1570723
det: 4.2097550226444183e+27
[5.65626150e+08 5.74137495e+05 3.81349256e+05 2.62088718e+04
 1.29700068e+03]
Pyomo OF: 566609142.1570723
Log_det: 63.60720196744483
num_static_measurement: 72
Tg_0_1_des :  0.3
vg_0_1_des :  1.0
y_CO2_0_0_ads
[1.]
y_CO2_1_0_ads
[1.]
y_CO2_2_0_ads
[1.]
y_CO2_0_1_ads
[1.]
y_CO2_1_1_ads
[1.]
y_CO2_2_1_ads
[1.]
y_CO2_0_2_ads
[1.]
y_CO2_1_2_ads
[1.]
y_CO2_2_2_ads
[1.]
y_CO2_0_0_des
[1.]
y_CO2_1_0_des
[1.]
y_CO2_2_0_des
[1.]
y_CO2_0_1_des
[1.]
y_CO2_1_1_des
[1.]
y_CO2_2_1_des
[1.]
y_CO2_0_2_des
[1.]
y_CO2_1_2_des
[1.]
y_

In [49]:
store = True

if store:
    file = open('Sep17_'+str(budget_opt)+'_a', 'wb')

    pickle.dump(ans_y, file)

    file.close()
    
    file2 = open('Sep17_fim_'+str(budget_opt)+'_a', 'wb')

    pickle.dump(fim_result, file2)

    file2.close()

### verify 

In [ ]:
## rank FIMs 

measure_names = [
    'Tg_0', 'Tg_1', 'Tg_2', 'Pg_0', 'Pg_1', 'Pg_2', 'vg_0', 'vg_1', 'vg_2', 
    'Ts_0', 'Ts_1', 'Ts_2',
    'y_CO2_0', 'y_CO2_1', 'y_CO2_2', 'C_CO2_0', 'C_CO2_1', 
                 'C_CO2_2', 'C_N2_0', 'C_N2_1', 'C_N2_2', 
                 'qCO2_0', 'qCO2_1', 'qCO2_2',  'ytheta_0', 'ytheta_1', 'ytheta_2']

measure_lookup = {}

for i, n in enumerate(measure_names):
    measure_lookup[n] = i  


idx_lookup_solid = [1,5,10]
idx_lookup_gas = [5,20,40]

num_t = len(idx_lookup_solid)

In [ ]:
i=28

idx = i*num_total+i 

print(calculator.fim_collection[idx])

In [ ]:
### If it's computing right FIMs 

def verify_FIM(name, time_idx, ads=True): 
    
    if name in ['qCO2_0', 'qCO2_1', 'qCO2_2', 'Ts_0', 'Ts_1', 'Ts_2', 'ytheta_0', 'ytheta_1', 'ytheta_2']:
        lookup = idx_lookup_solid 
        
    else:
        lookup = idx_lookup_gas
       
    # index of this name+time index in MO package
    if ads:
        idx_converted = measure_lookup[name]*num_t*2+lookup.index(time_idx)
    else:
        idx_converted = measure_lookup[name]*num_t*2+3+lookup.index(time_idx) # 3: # rotational angle
    print("should be the ", idx_converted)
    
    # find computed FIMs 
    file = open("./MO_FIMs/"+name, "rb")
    data = pickle.load(file)
    file.close()
    
    print("calculated by precomuptation:")

    if ads:
        print(data["ads_"+str(time_idx)])
    else:
        print(data["des_"+str(time_idx)])
        
    #print(data)
        
    real_idx = idx_converted*num_total+idx_converted
        
    print("calculated by MO:", calculator.fim_collection[real_idx])
    
    

In [ ]:
verify_FIM("vg_2", 20, ads=False)

In [ ]:
verify_FIM("C_N2_2", 20, ads=False)

In [ ]:
### If it's computing right FIMs 

def rank_FIM(): 
    
    rank = {} # key: (name, time_idx, 0(ads)or1(des)),  # value: A-optimality 
    
    for name in measure_names:
        #print(name)
        if name in ['qCO2_0', 'qCO2_1', 'qCO2_2', 'Ts_0', 'Ts_1', 'Ts_2', 'ytheta_0', 'ytheta_1', 'ytheta_2']:
            lookup = idx_lookup_solid 

        else:
            lookup = idx_lookup_gas
            
        # find computed FIMs 
        file = open("./MO_FIMs/"+name, "rb")
        data = pickle.load(file)
        file.close()
            
        for real_idx in lookup:
            
            # 
            rank[(name, real_idx, 0)] = np.trace(data["ads_"+str(real_idx)])
            rank[(name, real_idx, 1)] = np.trace(data["des_"+str(real_idx)])
            
    return rank 
    


In [ ]:
ranking = rank_FIM()

In [ ]:
print(ranking)